# Классификация и градиентные спуски

В этой тетрадке мы попробуем немного посмотреть на то, как работают разные градиентные спуски. 

In [ ]:
from pathlib import Path
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch                     # Низкоуровневые штуки
import torch.nn as nn            # Высокоуровневые штуки
import torch.nn.functional as F  # Тоже высокоуровневые штуки, но с интерфейсом функций, а не классов

# 1. Выборка

Делать всё это мы будем на животных. Ежегодно около 7.6 миллионов бедных животных в США оказываются в приютах. Часть из них находит себе новую семью, часть возвращается к старому (бывает, что питомец потерялся и его нашли на улице), а часть погибает. Ужегодно усыпляется около 2.7 млн. собак и кошек.  

Используя датасет с входной информацией (цвет, пол, возраст и т.п.) из одного из приютов, мы попытаемся спрогнозировать что произойдёт с новыми животными, которые попадут в этот приют. Данные, используемые в тетрадке уже были предварительно обработаны и приведены в удобную для построения моделей форму. 

In [ ]:
X_path = Path('X_cat.csv')
y_path = Path('y_cat.csv')

if not X_path.exists():
    !wget -q https://github.com/dniku/neural_nets_dpo/raw/master/week04/X_cat.csv -O $X_path

if not y_path.exists():
    !wget -q https://github.com/dniku/neural_nets_dpo/raw/master/week04/y_cat.csv -O $y_path

In [ ]:
X = pd.read_csv('X_cat.csv', sep = '\t', index_col=0)
target = pd.read_csv('y_cat.csv', sep = '\t', index_col=0, header=None, names=['status'])['status']

print(X.shape)
X.head()

В датасете находится около 27 тысяч наблюдений и 37 фичей. Посмотрим на то как выглядит распределение того, что произошло со зверятами по особям.

In [ ]:
target.value_counts()

Видим, что классы несбалансированы. Попробуем оставить четыре класса и объединить класс умерших животных с классом животных, которых усыпили. 

In [ ]:
target[target == 'Died'] = 'Euthanasia'

target.value_counts()

Закодируем классы.

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(target)

print(dict(zip(range(len(le.classes_)), le.classes_)))
print(y)

Разобьём выборку на тренировочную и тестовую. 

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print(X_train.shape)
print(X_valid.shape)

Отшкалируем данные. Если это не сделать, модель будет учиться хуже.

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

X_train = ss.fit_transform(X_train)
X_valid = ss.transform(X_valid)

# 2. Архитектурка

Соберём что-то простенькое и более-менее с потолка.

In [ ]:
def make_new_model():
    dim_in = X_train.shape[1]
    dim_out = len(le.classes_)
    
    model = <YOUR CODE>
    
    return model

print(make_new_model())

# 3. Данные

Завернём датасет в даталоадер, чтобы бесплатно получить перемешивание и нарезку на батчи.

In [ ]:
def prepare_data(X, y, batch_size=1000, shuffle=False):
    X_torch = torch.tensor(X.values if isinstance(X, pd.DataFrame) else X, dtype=torch.float32)
    y_torch = torch.tensor(y, dtype=torch.int64)
    
    dataset = torch.utils.data.TensorDataset(X_torch, y_torch)
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    
    return X_torch, y_torch, dataset, loader

X_train_torch, y_train_torch, train_dataset, train_loader = prepare_data(X_train, y_train, shuffle=True)
X_valid_torch, y_valid_torch, valid_dataset, valid_loader = prepare_data(X_valid, y_valid)

# 4. Training loop

Решаем задачу классификации, поэтому в качестве лосса берём кросс-энтропию. Если будет время, в следующий раз напишем `CrossEntropyLoss` самостоятельно.

In [ ]:
criterion = nn.CrossEntropyLoss()

Всё как в прошлый раз.

In [ ]:
def train(model, opt, criterion, train_loader, valid_loader, num_epochs):
    # Заводим словарь, куда будем писать логи
    history = {'loss_train': [], 'loss_valid': []}
    
    with tqdm(range(num_epochs)) as progress_bar:
        for epoch in progress_bar:  # цикл по эпохам
            epoch_losses_train = []
            epoch_losses_valid = []

            for X_batch, y_batch in train_loader:  # цикл по train-датасету (его за нас перемешивает DataLoader)
                logits = <YOUR CODE>
                loss = <YOUR CODE>
                
                # Посчитайте градиенты и сделайте шаг оптимизатора (он передаётся в функцию параметром opt)
                <YOUR CODE>

                # .item() конвертирует тензор из одного элемента в питоновское число
                epoch_losses_train.append(loss.item())

            with torch.no_grad():  # отключаем построение вычислительного графа на время валидации
                for X_batch, y_batch in valid_loader:  # цикл по valid-датасету
                    logits = <YOUR CODE>
                    loss = <YOUR CODE>

                    epoch_losses_valid.append(loss.item())
            
            # Записываем логи
            history['loss_train'].append(np.mean(epoch_losses_train))
            history['loss_valid'].append(np.mean(epoch_losses_valid))
            
            # Обновляем прогресс-бар
            progress_bar.set_postfix_str(
                f'Train loss: {history["loss_train"][-1]:.3f}, ' +
                f'Validation loss: {history["loss_valid"][-1]:.3f}')

    return history

# 5. Эксперименты!

In [ ]:
num_epochs = 500
learning_rate = 0.01

In [ ]:
histories = {}

Функция для графиков:

In [ ]:
def plot_history(histories):
    plt.figure(figsize=(16, 10))

    for name, history in histories.items():
        train = plt.plot(history['loss_train'], label=f'{name} train')
        plt.plot(history['loss_valid'], color=train[0].get_color(), linestyle='--', label=f'{name} valid')

    plt.xlabel('Epochs')
    plt.ylabel('Log loss')
    plt.legend()
    plt.grid()

### SGD 

Как и раньше, используем класс `torch.optim.SGD`.

In [ ]:
model = make_new_model()
opt = <YOUR CODE>
histories['SGD'] = train(model, opt, criterion, train_loader, valid_loader, num_epochs)

In [ ]:
plot_history(histories)

### SGD with momentum

Снова используем `torch.optim.SGD`, но указываем параметр `momentum`.

In [ ]:
model = make_new_model()
opt = <YOUR CODE>
histories['Momentum'] = train(model, opt, criterion, train_loader, valid_loader, num_epochs)

In [ ]:
plot_history(histories)

### RMSprop 

Понадобится класс `torch.optim.RMSprop`.

In [ ]:
model = make_new_model()
opt = <YOUR CODE>
histories['RMSProp'] = train(model, opt, criterion, train_loader, valid_loader, num_epochs)

In [ ]:
plot_history(histories)

### Adam 

Наконец, попробуем `torch.optim.Adam`.

In [ ]:
model = make_new_model()
opt = <YOUR CODE>
histories['Adam'] = train(model, opt, criterion, train_loader, valid_loader, num_epochs)

In [ ]:
plot_history(histories)

# 6. Стратегии с постепенным понижением lr 

![](https://raw.githubusercontent.com/FUlyankin/neural_nets_econ/master/2019/sem_2/ahaha.jpg)

Попробуем уменьшать learning rate ступеньками: например, в 2 раза каждые 50 эпох. Для этого нам понадобится класс `torch.optim.lr_scheduler.StepLR`. Будем использовать его вместе с Адамом.

In [ ]:
# Добавили параметр scheduler
def train(model, opt, scheduler, criterion, train_loader, valid_loader, num_epochs):
    history = {'loss_train': [], 'loss_valid': [], 'lr': []}  # будем записывать lr
    with tqdm(range(num_epochs)) as progress_bar:
        for epoch in progress_bar:
            epoch_losses_train = []
            epoch_losses_valid = []

            for X_batch, y_batch in train_loader:
                logits = model(X_batch)
                loss = criterion(logits, y_batch)

                opt.zero_grad()
                loss.backward()
                opt.step()
                
                epoch_losses_train.append(loss.item())

            with torch.no_grad():
                for X_batch, y_batch in valid_loader:
                    logits = model(X_batch)
                    loss = criterion(logits, y_batch)

                    epoch_losses_valid.append(loss.item())
                    
            history['loss_train'].append(np.mean(epoch_losses_train))
            history['loss_valid'].append(np.mean(epoch_losses_valid))
            
            # вызываем scheduler
            scheduler.step()
            
            # записываем lr
            history['lr'].append(opt.param_groups[0]['lr'])

            progress_bar.set_postfix_str(
                f'Train loss: {history["loss_train"][-1]:.3f}, ' +
                f'Validation loss: {history["loss_valid"][-1]:.3f}, ' +
                f'LR: {history["lr"][-1]:.5f}')  # показываем lr

    return history


model = make_new_model()
opt = <YOUR CODE>
scheduler = <YOUR CODE>
histories['Adam + StepLR'] = train(model, opt, scheduler, criterion, train_loader, valid_loader, num_epochs)

In [ ]:
plot_history(histories)

In [ ]:
# функция для картинок, чтобы видеть как скорость обучения меняется от эпохи к эпохе
def plot_learning_rate(loss_history):
    epochs = len(loss_history)
    plt.plot(range(1, epochs + 1), loss_history, label='learning rate')
    plt.xlabel("epoch")
    plt.xlim([1, epochs + 1])
    plt.ylabel("learning rate")
    plt.legend(loc=0)
    plt.grid()

In [ ]:
plot_learning_rate(histories['Adam + StepLR']['lr'])

Попробуем ещё вариант: `torch.optim.lr_scheduler.ReduceLROnPlateau`. Этот класс умножает learning rate на параметр `factor`, когда в течение `patience` эпох лосс на валидации не уменьшается.

In [ ]:
def train(model, opt, scheduler, criterion, train_loader, valid_loader, num_epochs):
    history = {'loss_train': [], 'loss_valid': [], 'lr': []}
    with tqdm(range(num_epochs)) as progress_bar:
        for epoch in progress_bar:
            epoch_losses_train = []
            epoch_losses_valid = []

            for X_batch, y_batch in train_loader:
                logits = model(X_batch)
                loss = criterion(logits, y_batch)

                opt.zero_grad()
                loss.backward()
                opt.step()
                
                epoch_losses_train.append(loss.item())

            with torch.no_grad():
                for X_batch, y_batch in valid_loader:
                    logits = model(X_batch)
                    loss = criterion(logits, y_batch)

                    epoch_losses_valid.append(loss.item())
                    
            history['loss_train'].append(np.mean(epoch_losses_train))
            history['loss_valid'].append(np.mean(epoch_losses_valid))
                    
            scheduler.step(history['loss_valid'][-1])  # вызываем scheduler от валидационного лосса

            history['lr'].append(opt.param_groups[0]['lr'])

            progress_bar.set_postfix_str(
                f'Train loss: {history["loss_train"][-1]:.3f}, ' +
                f'Validation loss: {history["loss_valid"][-1]:.3f}, ' +
                f'LR: {history["lr"][-1]:.5f}')

    return history


model = make_new_model()
opt = <YOUR CODE>
scheduler = <YOUR CODE>
histories['Adam + ReduceLROnPlateau'] = train(
    model, opt, scheduler, criterion, train_loader, valid_loader, num_epochs)

In [ ]:
plot_history(histories)

In [ ]:
plot_learning_rate(histories['Adam + ReduceLROnPlateau']['lr'])

# 7. Что дальше?

* Другие расписания изменения скорости обучения (например, циклически меняющееся; для вдохновения см., например, [эту статью](https://www.jeremyjordan.me/nn-learning-rate/))
* Другая архитектура сетки (добавить слои или сделать глубже)
* ...

## Авторские права и почиташки 

Ноутбук основан на [ноутбуке](https://github.com/FUlyankin/neural_nets_dpo/blob/e296fc1/week03_grad/Keras_SGD_experiments_semisolve.ipynb) от Филиппа Ульянкина, который для его создания использовал [этот мануал](https://github.com/sukilau/Ziff-deep-learning/blob/master/3-CIFAR10-lrate/CIFAR10-lrate.ipynb).